In [2]:
import pandas as pd

# Beijing AQ

In [3]:
df1 = pd.read_csv('data/beijing_201802_201803_aq.csv')
df2 = pd.read_csv('data/beijing_17_18_aq.csv')
stations = list(df1.groupby(by="stationId").size().index)
l = []
for station in stations:
    new = df1[df1.stationId==station]
    old = df2[df2.stationId==station]
    l.extend([old, new])
beijing_aq = pd.concat(l)
beijing_aq["utc_time"] = pd.to_datetime(beijing_aq["utc_time"])
beijing_aq = beijing_aq.rename(columns={"stationId": "station_id"})

# London AQ

In [48]:
df1 = pd.read_csv('data/London_historical_aqi_forecast_stations_20180331.csv', index_col=0)
df2 = pd.read_csv('data/London_historical_aqi_other_stations_20180331.csv')
df2 = df2.drop(["Unnamed: 5", "Unnamed: 6"], axis=1)
df2 = df2.dropna(how="all")
df2 = df2[: -1]
df2 = df2.rename(columns={"Station_ID": "station_id"})
london_aq = pd.concat([df1, df2])
london_aq = london_aq.rename(columns={"MeasurementDateGMT": "utc_time"})
london_aq["utc_time"] = pd.to_datetime(london_aq["utc_time"])

/Users/wind/miniconda3/envs/python3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Beijing grid meteorology

In [68]:
df = pd.read_csv("data/Beijing_historical_meo_grid.csv")

In [71]:
beijing_meo = df.rename(columns={
    "stationName": "station_id",
    "utc_time": "time",
    "wind_speed/kph": "wind_speed",
})
beijing_meo["time"] = pd.to_datetime(beijing_meo["time"])

,station_id,longitude,latitude,time,temperature,pressure,humidity,wind_direction,wind_speed
0,beijing_grid_000,115.0,39.0,2017-01-01 00:00:00,-5.47,984.73,76.60,53.71,3.53
1,beijing_grid_001,115.0,39.1,2017-01-01 00:00:00,-5.53,979.33,75.40,43.59,3.11
2,beijing_grid_002,115.0,39.2,2017-01-01 00:00:00,-5.70,963.14,71.80,0.97,2.75
3,beijing_grid_003,115.0,39.3,2017-01-01 00:00:00,-5.88,946.94,68.20,327.65,3.84
4,beijing_grid_004,115.0,39.4,2017-01-01 00:00:00,-5.34,928.80,58.81,317.85,6.14
5,beijing_grid_005,115.0,39.5,2017-01-01 00:00:00,-4.81,910.66,49.43,313.46,8.52
6,beijing_grid_006,115.0,39.6,2017-01-01 00:00:00,-4.98,889.48,45.64,309.89,10.05
7,beijing_grid_007,115.0,39.7,2017-01-01 00:00:00,-5.49,866.77,44.66,306.63,11.16
8,beijing_grid_008,115.0,39.8,2017-01-01 00:00:00,-6.17,853.42,46.57,299.83,11.28
9,beijing_grid_009,115.0,39.9,2017-01-01 00:00:00,-7.17,858.79,54.29,281.74,9.98


# London grid meteorology

In [63]:
df = pd.read_csv("data/London_historical_meo_grid.csv")

In [66]:
london_meo = df.rename(columns={
    "stationName": "station_id",
    "utc_time": "time",
    "wind_speed/kph": "wind_speed"
})
london_meo["time"] = pd.to_datetime(london_meo["time"])

# APIs

In [103]:
import requests


def buildDataFrame(text):
    if len(text) < 10:
        return pd.DataFrame()
    tmp = text.split("\n")
    if (tmp[-1] == ""):
        tmp = tmp[: -1]
    for i in range(0, len(tmp)):
        if tmp[i][-1] == "\r":
            tmp[i] = tmp[i][: -1]
        tmp[i] = tmp[i].split(",")
    df = pd.DataFrame(tmp[1: ], columns=tmp[0])
    return df


def airQualityData(city="bj", year=2018, month=5, day=1, hour=0):
    if city == "bj":
        target = ["station_id", "time", "PM2.5", "PM10", "O3"]
    else:
        target = ["station_id", "time", "PM2.5", "PM10"]
    dic = {
        "PM25_Concentration": "PM2.5",
        "PM10_Concentration": "PM10",
        "O3_Concentration": "O3",
        "PM10 (ug/m3)": "PM10",
        "PM2.5 (ug/m3)": "PM2.5",
        "utc_time": "time",
    }
    if year == 2018 and month >= 4:
        url = 'https://biendata.com/competition/airquality/%s/%d-%d-%d-%d/%d-%d-%d-%d/2k0d1d8' % (city, year, month, day, hour, year, month, day, hour)
        response = requests.get(url)
        df = buildDataFrame(response.text)
        try:
            df["time"] = pd.to_datetime(df["time"])
        except:
            pass
    else:
        time = pd.Timestamp("%d-%d-%d %d:00:00" % (year, month, day, hour))
        if city == "bj":
            df = beijing_aq[beijing_aq["utc_time"] == time]
        else:
            df = london_aq[london_aq["utc_time"] == time]
        df = df.reset_index()
    
    df = df.rename(columns=dic)
    try:
        df = df[target]
    except:
        pass
    if df.empty:
        df = pd.DataFrame()
    return df


def meteorologyGridData(city="bj", year=2018, month=5, day=1, hour=0):
    target = ["station_id", "time", "temperature", "pressure",
              "humidity", "wind_direction", "wind_speed"]
    url = 'https://biendata.com/competition/meteorology/%s_grid/%d-%d-%d-%d/%d-%d-%d-%d/2k0d1d8' % (city, year, month, day, hour, year, month, day, hour)
    if year == 2018 and month >= 4:
        response = requests.get(url)
        df = buildDataFrame(response.text)
        try:
            df["time"] = pd.to_datetime(df["time"])
        except:
            pass
    else:
        time = pd.Timestamp("%d-%d-%d %d:00:00" % (year, month, day, hour))
        if city == "bj":
            df = beijing_meo[beijing_meo["time"] == time]
        else:
            df = london_meo[london_meo["time"] == time]
    try:
        df = df[target]
    except:
        pass
    df = df.reset_index(drop=True)
    if df.empty:
        df = pd.DataFrame()
    return df


def observedMeteorology(city="bj", year=2018, month=5, day=1, hour=0):
    if city != "bj":
        print("No observed meteorology data for London!!!!")
        return
    url = 'https://biendata.com/competition/meteorology/%s/%d-%d-%d-%d/%d-%d-%d-%d/2k0d1d8' % (city, year, month, day, hour, year, month, day, hour)
    response = requests.get(url)
    df = buildDataFrame(response.text)
    if df.empty:
        df = pd.DataFrame()
    return df

# airQualityData("ld", 2017, 4, 26, 2)
meteorologyGridData("ld", 2018, 4, 26, 2)

,station_id,time,temperature,pressure,humidity,wind_direction,wind_speed
0,london_grid_000,2018-04-26 02:00:00,9.8,1017.6761,75.0,275.69,38.18
1,london_grid_001,2018-04-26 02:00:00,8.72,1016.3848,78.0,273.29,32.53
2,london_grid_002,2018-04-26 02:00:00,7.64,1015.0936,81.0,269.88,26.97
3,london_grid_003,2018-04-26 02:00:00,6.61,1012.1083,85.0,265.81,22.84
4,london_grid_004,2018-04-26 02:00:00,5.65,1007.429,89.0,261.5,20.14
5,london_grid_005,2018-04-26 02:00:00,4.68,1002.7497,94.0,255.9,17.58
6,london_grid_006,2018-04-26 02:00:00,4.78,1001.8773,92.0,256.34,17.88
7,london_grid_007,2018-04-26 02:00:00,4.87,1001.0049,91.0,256.77,18.18
8,london_grid_008,2018-04-26 02:00:00,5.03,1001.1484,90.0,257.4,18.3
9,london_grid_009,2018-04-26 02:00:00,5.27,1002.3078,87.0,258.22,18.25
